**Account level data - US**

In [10]:
# Import packages, libraries and modules

#!pip install ds
import ds
import warnings
warnings.simplefilter("ignore")
import json
import os
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from ds.utilities.io import ds_trino
import numpy as np

# Display settings
from IPython.display import HTML, display
display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)

# Set max rows and columns to display
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.mode.chained_assignment = None

In [11]:
conn = ds_trino.create_trino_connection()

In [3]:
gmv_query ="""

with ordenes as (



with scan_out as ( SELECT
      CAST(order_uuid AS VARCHAR) AS order_id
      ,TRUE AS order_scanned_out
      ,count ((order_uuid)) as scanned_items
    FROM
     
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction 
    WHERE
      day_partition>= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '12' month, '%Y-%m-%d')
      
      AND type = 'TRANSACTION_ORDER_SCANOUT'
      AND cause = 'CAUSE_MANUAL'
    GROUP BY
      1,2)



SELECT 
DATE(from_unixtime(reference_time/1000, facility_timezone)) AS created_date,
--que empiece en domingo
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
date_trunc('week', DATE(from_unixtime((reference_time)/1000, facility_timezone))+ interval '1' day)- interval '1' day as created_week,
FLOOR((DATE_DIFF('day', DATE(created_at_ts), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 3) / 7) + 1 AS order_weeks_back_local,
cast(local_created_at_ts as time) as local_created_time,
facility_timezone,
'US/CAN' as region,
dc.country,



CASE 

when dc.country='United States' then 'USA'

else 'Other'

end as subregion,


(
CASE
   WHEN
      facility_id IN 
      (
         '2019e0f4-7fd8-47fd-a858-9e5c4b409a39',  -- DEN -  AURORA
         'ab647486-a823-456e-8058-053d052106de',   -- DEN - GRAPE
         '1e55ab2f-64a3-498c-a692-9614b79ab834'   -- DEN - VALLEJO
      )
   THEN
      'Denver'
   WHEN
      facility_id IN 
      (
         'b0629713-f1bd-4d2b-a065-46ff82645987',  -- SF - CHARTER OAK
         '7d248b5d-c113-4269-8f4a-68d155a48adf',  -- SF - SoMa
         '983bc311-51e8-4e2e-ba27-6cc94bec511c'  -- SF - Redwood City
      )
   THEN
      'San Francisco'
   WHEN
      facility_id IN 
      (
         '5d574e15-e1d4-42a3-837a-71eebe29728a',  -- OAK - ADELINE
         'bc9dcc91-8fec-4a62-8438-024a4f65e8e8'  -- OAK - JINGLETOWN
      )
   THEN
      'Oakland'
   WHEN
      facility_id IN 
      (
         '905cbe43-327f-4d3e-a897-48ee81a64ade'  -- SJO - Ruff (949 Ruff)
      )
   THEN
      'San Jose'
   WHEN
      facility_id IN 
      (
         '557194c1-5796-469c-b3e0-d83736880340'  -- SF - San Mateo (21st Street)
      )
   THEN
      'San Mateo'
   WHEN
      facility_id IN 
      (
         '9c7b97b6-878f-4007-9032-558c4c6157e9'  -- LA - Anaheim (Lewis)
      )
   THEN
      'Anaheim'
   WHEN
      facility_id IN 
      (
         'f0029277-0e7e-410a-ad4e-5247b892a280',  -- LA - Hollywood (Western)
         '99573ac0-a3f2-4c70-946c-e9c06b425d5c',  -- LA - Pico Union (1842 Washington)
         '11bcb365-1910-40e7-a25a-4d3dd15f6eb2',  -- LA - South Bay (Hawthorne)
         '5315a66f-8337-4c68-af42-86089a86f8ba',  -- LA - Long Beach (Daisy)
         '55a786a5-1a9d-4611-95e2-81009b44ebdb',  -- NA-US-CA(S)-LA-Pico-SA
         '9732b09c-9d18-4f8e-a5d4-34173ddf1ea4',  -- LA - Culver City (Selmaraine)
         '5efa5206-dc53-4ab4-9f49-e5a5a4dc1c34'  -- LA - University Park (W 38th)
      )
   THEN
      'Los Angeles'
   WHEN
      facility_id IN 
      (
         '4fff0653-ae90-4e98-a67d-f67001e33c65',  -- SD - College Area (El Cajon)
         'c5c4ed02-1518-4970-a68f-432c75ecf6e2'  -- SD - Downtown (Boston)
      )
   THEN
      'San Diego'
   WHEN
      facility_id IN 
      (
         '2fe8235d-fe85-46e1-be54-023a65e34939'  -- LAS - North Strip (St Louis)
      )
   THEN
      'Las Vegas'
   WHEN
      facility_id IN 
      (
         '2d661ae4-d4bf-4d88-8af1-2df4f7d0fd75'  -- AZ - Tucson (E Delano ST)
      )
   THEN
      'Tucson'
   WHEN
      facility_id IN 
      (
         '937ddb7e-4c76-4cfd-bbb8-bd07b33ee7e9',  -- PHX - Midtown (W Highland)
         'a9eb0fc1-7699-43b4-bd4c-6bb5b8195a87'  -- PHX - Tempe (E 5th)
      )
   THEN
      'Phoenix'
   WHEN
      facility_id IN 
      (
         '54341152-da81-4b58-8ba3-8604bc1ff2dc',  -- CHI - Avondale (Spaulding)
         '0f9ac987-32ac-4b5a-91bf-78cbc7ea44bb',  -- CHI - Humboldt Park (Grand)
         '75187d2a-01d8-467f-9993-90e200a3a586',  -- CHI - River West (Superior)
         '6ab373fb-ee91-4dc6-b529-ef67f6326588',  -- CHI - North Center (Rockwell)
         '94e1cb0b-facf-4f72-b66d-2bd97d98e58f'  -- CHI - South Loop (Wabash)
      )
   THEN
      'Chicago'
    WHEN
      facility_id IN 
      (
         '0a25ff7d-1697-4e0c-a394-71347b0cdcc3',  -- Minneapolis - Blaine
         '99174fc4-d9f4-424f-96d8-e499a85e85b6'  -- Minneapolis - Currie Avenue
      )
   THEN
      'Minneapolis'
   WHEN
      facility_id IN 
      (
         '4f7010ef-ba20-4a6a-927b-d1a37903ef5e'  -- OH - Cleveland - Central (Carnegie Ave)
      )
   THEN
      'Cleveland'
   WHEN
      facility_id IN 
      (
         '6936d9f5-887a-4753-9c77-029c0164d956'  -- COL - Short North (Essex)
      )
   THEN
      'Columbus'
   WHEN
      facility_id IN 
      (
         'e1198539-7644-4c6d-9def-e64fa1cf4fb5',  -- MIA - South Beach (Alton)
         'd5f67e55-66be-4742-81c1-4720bb6906d8'  -- MIA - Wynwood (Miami Ct)
      )
   THEN
      'Miami'
   WHEN
      facility_id IN 
      (
         'd1814be1-e541-484d-9fff-9a4d800bfa26'  -- ORL - West Downtown (Dollins)
      )
   THEN
      'Orlando'
   WHEN
      facility_id IN 
      (
         'ba4cd06d-e256-4562-8ffc-372c7a2ead8c',  -- HOU - Midtown (Blodgett)
         '533a8386-c2a0-4e67-b6a3-ecea769ff764'  -- HOU - Galleria (Fairdale)
      )
   THEN
      'Houston'
   WHEN
      facility_id IN 
      (
         '68907756-0bb0-408f-a45c-3ab38083a57d',  -- DFW - Cullen (3004 Cullen)
         '2e30d25a-5c6b-46bf-b3f6-2030008dd5fd'  -- DFW - Towerwood (3230 Towerwood)
      )
   THEN
      'Fortworth'
   WHEN
      facility_id IN 
      (
         '4411e09a-c8b7-4a2e-ae6e-5d028e303791'  -- DAL - Trinity Groves (Commerce)
      )
   THEN
      'Dallas'
   WHEN
      facility_id IN 
      (
         'ee395d7f-589b-4208-a745-90a8a0aa3bfd'  -- San Antonio - Lombrano
      )
   THEN
      'San Antonio'
   WHEN
      facility_id IN 
      (
         'ce1e2ca1-6944-48d4-a93c-67bf011560f6'  -- ATX - North Loop (Hwy 35)
      )
   THEN
      'Austin'
   WHEN
      facility_id IN 
      (
         '6f24178a-58f3-4d0a-a2c3-107ad22dcd4e',  -- ATL - Midtown (Forrest)
         '288c069f-0d07-4d56-8d10-dc8afab2cb94'  -- ATL - West End (Oak)
      )
   THEN
      'Georgia'
   WHEN
      facility_id IN 
      (
         '305dca7a-ffcf-4ecf-9363-4238af3cba6b',  -- PHL - University City (Fairmount)
         'efa863fe-4cf4-4552-b73d-253470878e8f'  -- PHL - North City Center (1308 W Girard)
      )
   THEN
      'Pennsylvania'
   WHEN
      facility_id IN 
      (
         'e92e292a-4ca8-4cbd-8b7d-82d275637c1f'  -- New Jersey - Belleville
      )
   THEN
      'New Jersey'
   WHEN
      facility_id IN 
      (
         'e380140f-dfa9-4b17-84aa-61af20324195'  -- PROV - Smith Hill (Bath)
      )
   THEN
      'Providence'
   WHEN
      facility_id IN 
      (
         '6a896a7e-ad07-4f06-87e9-1af5ea87d6cb',  -- NYC - Long Island City
         '65820060-3a5f-4561-9f7a-9a7e34c581c1',  -- NYC - Brooklyn (Utica)
         '9854b8e1-8c33-4ee0-b6b5-4ccbc2012a38',  -- NYC - Astoria (24-26 47th)
         '9f5e4667-6a63-4af6-a805-476c979884e2',  -- NY - The Bronx (Bullard Ave)
         '9209f242-c27f-4174-b941-73e4f7b00e56',  -- NA-US-NY-NYC-Broadway 
         '05db32ee-158c-4baa-97c6-402bba3fcad3',  -- NA-US-NY-NYC-Church
         '628e10de-20b0-400b-a006-8e023e4f5c4d',  -- NA-US-NY-Brooklyn-Commerce
         '7d62cf8a-dfc8-4988-a02e-0b27303b19e2'  -- NA-US-NY-NYC-Devoe
      )
   THEN
      'New York City'
   WHEN
      facility_id IN 
      (
         '8365d2e5-dd91-4a87-beaa-cb54c374acdc'  -- NA-US-NY-BUF-Military
      )
   THEN
      'Buffalo'
    WHEN
      facility_id IN 
      (
         '54d4375f-4c79-4444-ad83-f54362c08198'  -- NA-US-DC-5th
      )
   THEN
      'DC'
END
) AS city,

(
CASE
   WHEN facility_id = '2019e0f4-7fd8-47fd-a858-9e5c4b409a39' THEN 'NA-US-CO-DEN-Aurora'
   WHEN facility_id = 'ab647486-a823-456e-8058-053d052106de' THEN 'NA-US-CO-DEN-GRAPE'
   WHEN facility_id = '1e55ab2f-64a3-498c-a692-9614b79ab834' THEN 'NA-US-CO-DEN-Vallejo'
   
   WHEN facility_id = 'b0629713-f1bd-4d2b-a065-46ff82645987' THEN 'NA-US-CA(N)-SF-Charter Oak'
   WHEN facility_id = '7d248b5d-c113-4269-8f4a-68d155a48adf' THEN 'NA-US-CA(N)-SF-SoMa'
   WHEN facility_id = '983bc311-51e8-4e2e-ba27-6cc94bec511c' THEN 'NA-US-CA(N)-SF-Macarthur (Redwood City)'

   WHEN facility_id = '5d574e15-e1d4-42a3-837a-71eebe29728a' THEN 'NA-US-CA(N)-OAK-Adeline'
   WHEN facility_id = 'bc9dcc91-8fec-4a62-8438-024a4f65e8e8' THEN 'NA-US-CA(N)-OAK-Jingletown'

   WHEN facility_id = '905cbe43-327f-4d3e-a897-48ee81a64ade' THEN 'NA-US-CA(N)-SJ-Ruff (Kitchens)'

   WHEN facility_id = '557194c1-5796-469c-b3e0-d83736880340' THEN 'NA-US-CA(N)-SM-21st'

   WHEN facility_id = '9c7b97b6-878f-4007-9032-558c4c6157e9' THEN 'NA-US-CA(S)-COA-Lewis'

   WHEN facility_id = 'f0029277-0e7e-410a-ad4e-5247b892a280' THEN 'NA-US-CA(S)-LA-Hollywood'
   WHEN facility_id = '99573ac0-a3f2-4c70-946c-e9c06b425d5c' THEN 'NA-US-CA(S)-LA-Koreatown'
   WHEN facility_id = '11bcb365-1910-40e7-a25a-4d3dd15f6eb2' THEN 'NA-US-CA(S)-LAW-Hawthorne'
   WHEN facility_id = '5315a66f-8337-4c68-af42-86089a86f8ba' THEN 'NA-US-CA(S)-LA-LongBeach'
   WHEN facility_id = '55a786a5-1a9d-4611-95e2-81009b44ebdb' THEN 'NA-US-CA(S)-LA-Pico-SA'
   WHEN facility_id = '9732b09c-9d18-4f8e-a5d4-34173ddf1ea4' THEN 'NA-US-CA(S)-CC-Selmaraine'
   WHEN facility_id = '5efa5206-dc53-4ab4-9f49-e5a5a4dc1c34' THEN 'NA-US-CA(S)-LA-UniversityPark'

   WHEN facility_id = '4fff0653-ae90-4e98-a67d-f67001e33c65' THEN 'NA-US-CA(S)-SD-El Cajon'
   WHEN facility_id = 'c5c4ed02-1518-4970-a68f-432c75ecf6e2' THEN 'NA-US-CA(S)-SD-Downtown'

   WHEN facility_id = '2fe8235d-fe85-46e1-be54-023a65e34939' THEN 'NA-US-NV-LAS-North Strip'

   WHEN facility_id = '2d661ae4-d4bf-4d88-8af1-2df4f7d0fd75' THEN 'AZ - Tucson (E Delano ST)'

   WHEN facility_id = '937ddb7e-4c76-4cfd-bbb8-bd07b33ee7e9' THEN 'NA-US-AZ-PHX-Highland (Kitchens)'
   WHEN facility_id = 'a9eb0fc1-7699-43b4-bd4c-6bb5b8195a87' THEN 'NA-US-AZ-PHX-Tempe (Kitchens)'

   WHEN facility_id = '54341152-da81-4b58-8ba3-8604bc1ff2dc' THEN 'NA-US-IL-CHI-Avondale'
   WHEN facility_id = '0f9ac987-32ac-4b5a-91bf-78cbc7ea44bb' THEN 'NA-US-IL-CHI-Humboldt Park'
   WHEN facility_id = '75187d2a-01d8-467f-9993-90e200a3a586' THEN 'NA-US-IL-CHI-River West'
   WHEN facility_id = '6ab373fb-ee91-4dc6-b529-ef67f6326588' THEN 'NA-US-IL-CHI-Rockwell'
   WHEN facility_id = '94e1cb0b-facf-4f72-b66d-2bd97d98e58f' THEN 'NA-US-IL-CHI-South Loop'

   WHEN facility_id = '0a25ff7d-1697-4e0c-a394-71347b0cdcc3' THEN 'NA-US-MN-MPLS-Blaine'
   WHEN facility_id = '99174fc4-d9f4-424f-96d8-e499a85e85b6' THEN 'NA-US-MN-MPLS-Currie'

   WHEN facility_id = '4f7010ef-ba20-4a6a-927b-d1a37903ef5e' THEN 'NA-US-OH-Carnegie'

   WHEN facility_id = '6936d9f5-887a-4753-9c77-029c0164d956' THEN 'NA-US-OH-COL-ShortNorth (Kitchens)'
   
   WHEN facility_id = 'e1198539-7644-4c6d-9def-e64fa1cf4fb5' THEN 'NA-US-FL-MIA-Alton'
   WHEN facility_id = 'd5f67e55-66be-4742-81c1-4720bb6906d8' THEN 'ORL - West Downtown (Dollins)' -- pending SFDC Name

   WHEN facility_id = 'd1814be1-e541-484d-9fff-9a4d800bfa26' THEN 'NA-US-FL-ORL-Dollins'

   WHEN facility_id = 'ba4cd06d-e256-4562-8ffc-372c7a2ead8c' THEN 'NA-US-TX-HOU-Blodgett (Kitchen)'
   WHEN facility_id = '533a8386-c2a0-4e67-b6a3-ecea769ff764' THEN 'NA-US-TX-HOU-Fairdale (Kitchens)'

   WHEN facility_id = '68907756-0bb0-408f-a45c-3ab38083a57d' THEN 'NA-US-TX-FortWorth-CullenSt'
   WHEN facility_id = '2e30d25a-5c6b-46bf-b3f6-2030008dd5fd' THEN 'DFW - Towerwood (3230 Towerwood)' -- pending SFDC Name

   WHEN facility_id = '4411e09a-c8b7-4a2e-ae6e-5d028e303791' THEN 'NA-US-TX-DAL-Commerce'

   WHEN facility_id = 'ee395d7f-589b-4208-a745-90a8a0aa3bfd' THEN 'NA-US-TX-SA-Lombrano'

   WHEN facility_id = 'ce1e2ca1-6944-48d4-a93c-67bf011560f6' THEN 'NA-US-TX-AUS-North Loop'

   WHEN facility_id = '6f24178a-58f3-4d0a-a2c3-107ad22dcd4e' THEN 'NA-US-GA-ATL-Forrest (Kitchens)'
   WHEN facility_id = '288c069f-0d07-4d56-8d10-dc8afab2cb94' THEN 'NA-US-GA-ATL-Oak'

   WHEN facility_id = '305dca7a-ffcf-4ecf-9363-4238af3cba6b' THEN 'NA-US-PA-PHL-Fairmount'
   WHEN facility_id = 'efa863fe-4cf4-4552-b73d-253470878e8f' THEN 'NA-US-PA-PHL-Girard'

   WHEN facility_id = 'e92e292a-4ca8-4cbd-8b7d-82d275637c1f' THEN 'NA-US-NJ-Cortlandt'

   WHEN facility_id = 'e380140f-dfa9-4b17-84aa-61af20324195' THEN 'NA-US-RI-PROV-Smithhill'

   WHEN facility_id = '6a896a7e-ad07-4f06-87e9-1af5ea87d6cb' THEN 'NA-US-NY-LIC-Skillman'
   WHEN facility_id = '65820060-3a5f-4561-9f7a-9a7e34c581c1' THEN 'NA-US-NY-NYC-Utica'
   WHEN facility_id = '9854b8e1-8c33-4ee0-b6b5-4ccbc2012a38' THEN 'NA-US-IL-CHI-River West'
   WHEN facility_id = '9f5e4667-6a63-4af6-a805-476c979884e2' THEN 'NY - The Bronx (Bullard Ave)' -- pending SFDC Name
   WHEN facility_id = '9209f242-c27f-4174-b941-73e4f7b00e56' THEN 'NA-US-NY-NYC-Broadway'
   WHEN facility_id = '05db32ee-158c-4baa-97c6-402bba3fcad3' THEN 'NA-US-NY-NYC-Church'
   WHEN facility_id = '628e10de-20b0-400b-a006-8e023e4f5c4d' THEN 'NA-US-NY-Brooklyn-Commerce'
   WHEN facility_id = '7d62cf8a-dfc8-4988-a02e-0b27303b19e2' THEN 'NA-US-NY-NYC-Devoe'

   WHEN facility_id = '8365d2e5-dd91-4a87-beaa-cb54c374acdc' THEN 'NA-US-NY-BUF-Military'

   WHEN facility_id = '54d4375f-4c79-4444-ad83-f54362c08198' THEN 'NA-US-DC-5th'

END
) AS facility_sf_name,
facility_name,
organization_name as org_name,
organization_id,
co.order_id,
facility_id,
order_display_id,
ofo_slug, store_link_service_slug,
facility_country_code as country_code, 
is_cancelled,
oc_order_issue_count,
order_issue_details,
hoi_order_issue_details,
oc_order_issue_details,
order_issue_types,
hoi_order_issue_types,
oc_order_issue_types,

subtotal,
total,

commission, discount,	tip, tip_for_restaurant, tip_for_courier, delivery_fee, marketing_fee, service_fee

,ROW_NUMBER() OVER (PARTITION BY ofo_slug, external_order_id, order_items_quantity, subtotal ORDER BY co.order_id) as row_number

,cast(local_accepted_at_last_ts as TIME ) as accepted_local_time,
local_processed_at_ts,local_ordered_at_ts,local_accepted_at_ts, local_in_handoff_at_ts,
local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,local_pickup_picked_up_at_ts,
local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,fulfillment_state,fulfillment_mode,
pickup_state,scheduling_type,ofo_status,order_status
,cancellation_state,cancellation_reason as facility_cancellation_reason
,ofo_cancellation_reason,order_rating, order_review_full_text,location_state,

case when location_state='LOCATION_HANDED_OFF' then 1
  else 0
  end
as is_handed_off_num,

Case When in_handoff_at IS NOT NULL THEN 1
  else 0
  END 
as is_in_handoff_num,




CASE
        WHEN is_cancelled THEN 'cancel'
        WHEN order_issue_types IS NOT NULL THEN 'issue'
        ELSE 'success'
        END AS defect_type,

CASE
        WHEN is_cancelled AND cancellation_reason = 'UNAVAILABLE_ITEM' THEN 'item_unavailable'
        WHEN is_cancelled AND confirmation_state <> 'CONFIRMATION_CONFIRMED' THEN 'confirmation_failed'
        WHEN is_cancelled AND  location_state!='LOCATION_HANDED_OFF' THEN 'handoff_failed'
        WHEN is_cancelled AND time_to_last_in_handoff_millis > 1200000 THEN 'pick_over_20' --Pick time was over 20 minutes long
        WHEN is_cancelled THEN 'other_cancel'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ORDER%' THEN 'order_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%MISSING_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_QUANTITY%' THEN 'item_missing'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_SIZE%' THEN 'item_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%INCOMPLETE%' THEN 'order_incomplete'
        ELSE 'other_issue'
      END AS defect_reason,

COALESCE(IF(adjustment IS NOT NULL AND (is_cancelled OR order_issue_types IS NOT NULL), adjustment < 0), FALSE) AS is_refunds
,time_to_last_in_handoff_millis BETWEEN 1 AND 1200000 AS order_20picked --TRUE if picked in under 20 minutes

,order_source, hoi_only_order_type,order_items_quantity,scan_out.order_scanned_out,


printed_at_ts, in_handoff_at_ts, handed_off_at_ts, courier_checked_in_at_ts, courier_dropped_off_at_ts, courier_arrived_at_ts

,scanned_items

,customer_note

,concat('https://tools.cloudkitchens.com/order-inspector/order-details/',co.order_id,'/overview') as link


FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'USC'
   
left join scan_out
on co.order_id=scan_out.order_id






--aqui se filtra la fecha    
WHERE co.day_partition >= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '12' month, '%Y-%m-%d')

AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-15'
AND tenant_type = 'VIRTUAL'

--AND is_cancelled

AND facility_id IN ( -- 
-- COLORADO
    -- DENVER
           '2019e0f4-7fd8-47fd-a858-9e5c4b409a39',  -- DEN -  AURORA
           'ab647486-a823-456e-8058-053d052106de',   -- DEN - GRAPE
           '1e55ab2f-64a3-498c-a692-9614b79ab834',   -- DEN - VALLEJO

-- CALIFORNIA
    -- SAN FRANCISCO
           'b0629713-f1bd-4d2b-a065-46ff82645987',  -- SF - CHARTER OAK
           '7d248b5d-c113-4269-8f4a-68d155a48adf',  -- SF - SoMa
           '983bc311-51e8-4e2e-ba27-6cc94bec511c',  -- SF - Redwood
    -- OAKLAND
           '5d574e15-e1d4-42a3-837a-71eebe29728a',  -- OAK - Adeline
           'bc9dcc91-8fec-4a62-8438-024a4f65e8e8',  -- OAK - Jingletown (12th St)
   -- SAN JOSE
           '905cbe43-327f-4d3e-a897-48ee81a64ade',  -- SJO - Ruff (949 Ruff)
   -- SAN MATEO
           '557194c1-5796-469c-b3e0-d83736880340',  -- SF - San Mateo (21st Street)
   -- ANAHEIM
           '9c7b97b6-878f-4007-9032-558c4c6157e9',  -- LA - Anaheim (Lewis)
   -- LOS ANGELES
           'f0029277-0e7e-410a-ad4e-5247b892a280',  -- LA - Hollywood (Western)
           '99573ac0-a3f2-4c70-946c-e9c06b425d5c',  -- LA - Pico Union (1842 Washington)
           '11bcb365-1910-40e7-a25a-4d3dd15f6eb2',  -- LA - South Bay (Hawthorne)
           '5315a66f-8337-4c68-af42-86089a86f8ba',  -- LA - Long Beach (Daisy)
           '55a786a5-1a9d-4611-95e2-81009b44ebdb',  -- NA-US-CA(S)-LA-Pico-SA
           '9732b09c-9d18-4f8e-a5d4-34173ddf1ea4',  -- LA - Culver City (Selmaraine)
           '5efa5206-dc53-4ab4-9f49-e5a5a4dc1c34',  -- LA - University Park (W 38th)
    -- SAN DIEGO
           '4fff0653-ae90-4e98-a67d-f67001e33c65',  -- SD - College Area (El Cajon)
           'c5c4ed02-1518-4970-a68f-432c75ecf6e2',  -- SD - Downtown (Boston)
-- NEVADA
   -- LAS VEGAS
           '2fe8235d-fe85-46e1-be54-023a65e34939',  -- LAS - North Strip (St Louis)
-- ARIZONA
   -- TUCSON
           '2d661ae4-d4bf-4d88-8af1-2df4f7d0fd75',  -- AZ - Tucson (E Delano ST)
   -- PHOENIX
           '937ddb7e-4c76-4cfd-bbb8-bd07b33ee7e9',  -- PHX - Midtown (W Highland)
           'a9eb0fc1-7699-43b4-bd4c-6bb5b8195a87',  -- PHX - Tempe (E 5th)
-- ILLINOIS
   -- TUCSON
           '54341152-da81-4b58-8ba3-8604bc1ff2dc',  -- CHI - Avondale (Spaulding)
           '0f9ac987-32ac-4b5a-91bf-78cbc7ea44bb',  -- CHI - Humboldt Park (Grand)
           '75187d2a-01d8-467f-9993-90e200a3a586',  -- CHI - River West (Superior)
           '6ab373fb-ee91-4dc6-b529-ef67f6326588',  -- CHI - North Center (Rockwell)
           '94e1cb0b-facf-4f72-b66d-2bd97d98e58f',  -- CHI - South Loop (Wabash)
-- MINNESOTA
   -- MINNEAPOLIS
           '0a25ff7d-1697-4e0c-a394-71347b0cdcc3',  -- Minneapolis - Blaine
           '99174fc4-d9f4-424f-96d8-e499a85e85b6',  -- Minneapolis - Currie Avenue
-- OHIO
   -- CLEVELAND
           '4f7010ef-ba20-4a6a-927b-d1a37903ef5e',  -- OH - Cleveland - Central (Carnegie Ave)
   -- COLUMBUS
           '6936d9f5-887a-4753-9c77-029c0164d956',  -- COL - Short North (Essex)
-- FLORIDA
   -- MIAMI
           'e1198539-7644-4c6d-9def-e64fa1cf4fb5',  -- MIA - South Beach (Alton)
           'd5f67e55-66be-4742-81c1-4720bb6906d8',  -- MIA - Wynwood (Miami Ct)
   -- ORLANDO
           'd1814be1-e541-484d-9fff-9a4d800bfa26',  -- ORL - West Downtown (Dollins)
-- TEXAS
   -- HOUSTON
           'ba4cd06d-e256-4562-8ffc-372c7a2ead8c',  -- HOU - Midtown (Blodgett)
           '533a8386-c2a0-4e67-b6a3-ecea769ff764',  -- HOU - Galleria (Fairdale)
   -- FORTWORTH
           '68907756-0bb0-408f-a45c-3ab38083a57d',  -- DFW - Cullen (3004 Cullen)
           '2e30d25a-5c6b-46bf-b3f6-2030008dd5fd',  -- DFW - Towerwood (3230 Towerwood)
   -- DALLAS
           '4411e09a-c8b7-4a2e-ae6e-5d028e303791',  -- DAL - Trinity Groves (Commerce)
   -- SAN ANTONIO
           'ee395d7f-589b-4208-a745-90a8a0aa3bfd',  -- San Antonio - Lombrano
   -- AUSTIN
           'ce1e2ca1-6944-48d4-a93c-67bf011560f6',  -- ATX - North Loop (Hwy 35)
-- ATLANTA
   -- GEORGIA
           '6f24178a-58f3-4d0a-a2c3-107ad22dcd4e',  -- ATL - Midtown (Forrest)
           '288c069f-0d07-4d56-8d10-dc8afab2cb94',  -- ATL - West End (Oak)
-- PHILADELPHIA
   -- PENNSYLVANIA
           '305dca7a-ffcf-4ecf-9363-4238af3cba6b',  -- PHL - University City (Fairmount)
           'efa863fe-4cf4-4552-b73d-253470878e8f',  -- PHL - North City Center (1308 W Girard)
-- NEW JERSEY
   -- NEW JERSEY
           'e92e292a-4ca8-4cbd-8b7d-82d275637c1f',  -- New Jersey - Belleville
-- RHODE ISLAND
   -- PROVIDENCE
           'e380140f-dfa9-4b17-84aa-61af20324195',  -- PROV - Smith Hill (Bath)
-- NEW YORK
    -- NEW YORK CITY
           '6a896a7e-ad07-4f06-87e9-1af5ea87d6cb',  -- NYC - Long Island City
           '65820060-3a5f-4561-9f7a-9a7e34c581c1',  -- NYC - Brooklyn (Utica)
           '9854b8e1-8c33-4ee0-b6b5-4ccbc2012a38',  -- NYC - Astoria (24-26 47th)
           '9f5e4667-6a63-4af6-a805-476c979884e2',  -- NY - The Bronx (Bullard Ave)
           '9209f242-c27f-4174-b941-73e4f7b00e56',  -- NA-US-NY-NYC-Broadway
           '05db32ee-158c-4baa-97c6-402bba3fcad3',  -- NA-US-NY-NYC-Church
           '628e10de-20b0-400b-a006-8e023e4f5c4d',  -- NA-US-NY-Brooklyn-Commerce
           '7d62cf8a-dfc8-4988-a02e-0b27303b19e2',  -- NA-US-NY-NYC-Devoe
    -- BUFFALO
           '8365d2e5-dd91-4a87-beaa-cb54c374acdc',  -- NA-US-NY-BUF-Military
-- WASHINGTON DC
    -- WASHINGTON DC
           '54d4375f-4c79-4444-ad83-f54362c08198'  -- NA-US-DC-5th

                )


)


Select created_month,country,region,subregion,city, org_name, ordenes.organization_id

,CASE 
when org.deleted_at is not null then True

ELSE  False END

as org_deleted_skusku





, count (DISTINCT (order_id)) as total_orders
, count(DISTINCT CASE WHEN is_cancelled THEN order_id END) as cancelled_orders
,count (DISTINCT CASE WHEN defect_type = 'issue' THEN order_id END) as issue_orders
,count (DISTINCT CASE WHEN defect_type != 'success' THEN order_id END) as ODR_orders
,count(DISTINCT  CASE WHEN order_scanned_out THEN order_id END) as scanned_orders
,sum(subtotal) as subtotal_local


,sum(total) as total_local
,sum(commission) as commission_local

 ,sum(discount) as discount_local


,SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS cancelled_gmv_local
,count (DISTINCT ofo_slug) as OFOs
,count(DISTINCT city) as cities
, count(DISTINCT facility_sf_name) as facilities
, AVG (order_items_quantity) as Avg_Items_per_Order
, AVG(order_rating) as avg_rating
, AVG(in_handoff_at_ts-printed_at_ts) as avg_picking_time
,  avg(date_diff('minute',printed_at_ts,in_handoff_at_ts) )as avg_picking_time_minutes 
from ordenes

left join hudi_ingest.processed.store_config_organization_view as org

on ordenes.organization_id=org.organization_id

WHERE  row_number=1

group by 1,2,3,4,5,6,7,8


"""
print("Querying data...")
gmv_data = ds_trino.fetch_data(conn=conn, sql_str= gmv_query, use_cache=False)
print("Data pulled!")

Querying data...
Data pulled!


In [4]:
gmv_data.head()

,created_month,country,region,subregion,city,org_name,organization_id,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,subtotal_local,total_local,commission_local,discount_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes
0,2022-01-01,United States,US/CAN,USA,Los Angeles,Pink Dot,8a0b24e8-605f-4071-a2e3-42b6f23a8190,False,107,14,2,16,2,2923.46,2863.80,-402.10,-310.55,333.77,2,1,1,1.700935,4.500000,0 00:06:33.152,6.101266
1,2022-05-01,United States,US/CAN,USA,Chicago,"Eddie G's Cookies, INC",74ffdcf6-92e5-4f69-b2ea-57f3681659ac,False,8,0,0,0,0,109.83,114.20,-24.27,-4.99,0.00,2,1,2,2.125000,1.000000,0 00:04:29.125,4.125000
2,2022-06-01,United States,US/CAN,USA,San Francisco,Cacao America LLC,1ee4654e-5c30-49f1-b82f-6107d7d8ead9,False,58,4,6,10,0,2794.98,2767.05,-216.25,-140.33,442.30,2,1,1,3.551724,5.000000,0 00:07:47.432,7.405405
3,2022-12-01,United States,US/CAN,USA,Los Angeles,MTFST LLC,5d44b786-f55a-48ac-bc27-972c5c3ab327,False,32,1,1,2,20,991.00,648.13,-32.88,-357.58,18.00,4,1,1,4.062500,NaN,None,NaN
4,2022-03-01,United States,US/CAN,USA,Columbus,Pink Dot,8a0b24e8-605f-4071-a2e3-42b6f23a8190,False,103,35,6,41,0,2759.46,2302.21,-314.11,-428.48,770.50,1,1,1,1.504854,4.583333,0 00:06:04.405,5.556962


In [5]:
print(len(gmv_data))

4859


In [6]:
gmv_data["city"].unique()

array(['Los Angeles', 'Chicago', 'San Francisco', 'Columbus', 'San Diego',
       'Austin', 'Oakland', 'Georgia', 'New York City', 'Pennsylvania',
       'San Jose', 'Phoenix', 'Anaheim', 'Houston', 'San Mateo', 'Denver',
       'Orlando', 'San Antonio', 'Providence', 'New Jersey', 'Fortworth',
       'Dallas', 'Las Vegas', 'Miami', 'Cleveland', 'Minneapolis', 'DC',
       'Buffalo'], dtype=object)

In [9]:
print("missing data as % of all data")
gmv_data.isna().sum() / len(gmv_data)

missing data as % of all data


created_month               0.000000
country                     0.000000
region                      0.000000
subregion                   0.000000
city                        0.000000
org_name                    0.000000
organization_id             0.000000
org_deleted_skusku          0.000000
total_orders                0.000000
cancelled_orders            0.000000
issue_orders                0.000000
ODR_orders                  0.000000
scanned_orders              0.000000
subtotal_local              0.000823
total_local                 0.000823
commission_local            0.092817
discount_local              0.067504
cancelled_gmv_local         0.000000
OFOs                        0.000000
cities                      0.000000
facilities                  0.000000
Avg_Items_per_Order         0.000000
avg_rating                  0.458531
avg_picking_time            0.270838
avg_picking_time_minutes    0.270838
dtype: float64

**Opportunity Data - US**

In [12]:
opty_query = """
     
with ordenes as (



with scan_out as ( SELECT
      CAST(order_uuid AS VARCHAR) AS order_id
      ,TRUE AS order_scanned_out
      ,count ((order_uuid)) as scanned_items
    FROM
     
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction 
    WHERE
      day_partition>= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '12' month, '%Y-%m-%d')
      
      AND type = 'TRANSACTION_ORDER_SCANOUT'
      AND cause = 'CAUSE_MANUAL'
    GROUP BY
      1,2)



SELECT 
DATE(from_unixtime(reference_time/1000, facility_timezone)) AS created_date,
--que empiece en domingo
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
date_trunc('week', DATE(from_unixtime((reference_time)/1000, facility_timezone))+ interval '1' day)- interval '1' day as created_week,
FLOOR((DATE_DIFF('day', DATE(created_at_ts), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 3) / 7) + 1 AS order_weeks_back_local,
cast(local_created_at_ts as time) as local_created_time,
facility_timezone,
'US/CAN' as region,
dc.country,




CASE 

when dc.country='United States' then 'USA'

else 'Other'

end as subregion,


(
CASE
   WHEN
      facility_id IN 
      (
         '2019e0f4-7fd8-47fd-a858-9e5c4b409a39',  -- DEN -  AURORA
         'ab647486-a823-456e-8058-053d052106de',   -- DEN - GRAPE
         '1e55ab2f-64a3-498c-a692-9614b79ab834'   -- DEN - VALLEJO
      )
   THEN
      'Denver'
   WHEN
      facility_id IN 
      (
         'b0629713-f1bd-4d2b-a065-46ff82645987',  -- SF - CHARTER OAK
         '7d248b5d-c113-4269-8f4a-68d155a48adf',  -- SF - SoMa
         '983bc311-51e8-4e2e-ba27-6cc94bec511c'  -- SF - Redwood City
      )
   THEN
      'San Francisco'
   WHEN
      facility_id IN 
      (
         '5d574e15-e1d4-42a3-837a-71eebe29728a',  -- OAK - ADELINE
         'bc9dcc91-8fec-4a62-8438-024a4f65e8e8'  -- OAK - JINGLETOWN
      )
   THEN
      'Oakland'
   WHEN
      facility_id IN 
      (
         '905cbe43-327f-4d3e-a897-48ee81a64ade'  -- SJO - Ruff (949 Ruff)
      )
   THEN
      'San Jose'
   WHEN
      facility_id IN 
      (
         '557194c1-5796-469c-b3e0-d83736880340'  -- SF - San Mateo (21st Street)
      )
   THEN
      'San Mateo'
   WHEN
      facility_id IN 
      (
         '9c7b97b6-878f-4007-9032-558c4c6157e9'  -- LA - Anaheim (Lewis)
      )
   THEN
      'Anaheim'
   WHEN
      facility_id IN 
      (
         'f0029277-0e7e-410a-ad4e-5247b892a280',  -- LA - Hollywood (Western)
         '99573ac0-a3f2-4c70-946c-e9c06b425d5c',  -- LA - Pico Union (1842 Washington)
         '11bcb365-1910-40e7-a25a-4d3dd15f6eb2',  -- LA - South Bay (Hawthorne)
         '5315a66f-8337-4c68-af42-86089a86f8ba',  -- LA - Long Beach (Daisy)
         '55a786a5-1a9d-4611-95e2-81009b44ebdb',  -- NA-US-CA(S)-LA-Pico-SA
         '9732b09c-9d18-4f8e-a5d4-34173ddf1ea4',  -- LA - Culver City (Selmaraine)
         '5efa5206-dc53-4ab4-9f49-e5a5a4dc1c34'  -- LA - University Park (W 38th)
      )
   THEN
      'Los Angeles'
   WHEN
      facility_id IN 
      (
         '4fff0653-ae90-4e98-a67d-f67001e33c65',  -- SD - College Area (El Cajon)
         'c5c4ed02-1518-4970-a68f-432c75ecf6e2'  -- SD - Downtown (Boston)
      )
   THEN
      'San Diego'
   WHEN
      facility_id IN 
      (
         '2fe8235d-fe85-46e1-be54-023a65e34939'  -- LAS - North Strip (St Louis)
      )
   THEN
      'Las Vegas'
   WHEN
      facility_id IN 
      (
         '2d661ae4-d4bf-4d88-8af1-2df4f7d0fd75'  -- AZ - Tucson (E Delano ST)
      )
   THEN
      'Tucson'
   WHEN
      facility_id IN 
      (
         '937ddb7e-4c76-4cfd-bbb8-bd07b33ee7e9',  -- PHX - Midtown (W Highland)
         'a9eb0fc1-7699-43b4-bd4c-6bb5b8195a87'  -- PHX - Tempe (E 5th)
      )
   THEN
      'Phoenix'
   WHEN
      facility_id IN 
      (
         '54341152-da81-4b58-8ba3-8604bc1ff2dc',  -- CHI - Avondale (Spaulding)
         '0f9ac987-32ac-4b5a-91bf-78cbc7ea44bb',  -- CHI - Humboldt Park (Grand)
         '75187d2a-01d8-467f-9993-90e200a3a586',  -- CHI - River West (Superior)
         '6ab373fb-ee91-4dc6-b529-ef67f6326588',  -- CHI - North Center (Rockwell)
         '94e1cb0b-facf-4f72-b66d-2bd97d98e58f'  -- CHI - South Loop (Wabash)
      )
   THEN
      'Chicago'
    WHEN
      facility_id IN 
      (
         '0a25ff7d-1697-4e0c-a394-71347b0cdcc3',  -- Minneapolis - Blaine
         '99174fc4-d9f4-424f-96d8-e499a85e85b6'  -- Minneapolis - Currie Avenue
      )
   THEN
      'Minneapolis'
   WHEN
      facility_id IN 
      (
         '4f7010ef-ba20-4a6a-927b-d1a37903ef5e'  -- OH - Cleveland - Central (Carnegie Ave)
      )
   THEN
      'Cleveland'
   WHEN
      facility_id IN 
      (
         '6936d9f5-887a-4753-9c77-029c0164d956'  -- COL - Short North (Essex)
      )
   THEN
      'Columbus'
   WHEN
      facility_id IN 
      (
         'e1198539-7644-4c6d-9def-e64fa1cf4fb5',  -- MIA - South Beach (Alton)
         'd5f67e55-66be-4742-81c1-4720bb6906d8'  -- MIA - Wynwood (Miami Ct)
      )
   THEN
      'Miami'
   WHEN
      facility_id IN 
      (
         'd1814be1-e541-484d-9fff-9a4d800bfa26'  -- ORL - West Downtown (Dollins)
      )
   THEN
      'Orlando'
   WHEN
      facility_id IN 
      (
         'ba4cd06d-e256-4562-8ffc-372c7a2ead8c',  -- HOU - Midtown (Blodgett)
         '533a8386-c2a0-4e67-b6a3-ecea769ff764'  -- HOU - Galleria (Fairdale)
      )
   THEN
      'Houston'
   WHEN
      facility_id IN 
      (
         '68907756-0bb0-408f-a45c-3ab38083a57d',  -- DFW - Cullen (3004 Cullen)
         '2e30d25a-5c6b-46bf-b3f6-2030008dd5fd'  -- DFW - Towerwood (3230 Towerwood)
      )
   THEN
      'Fortworth'
   WHEN
      facility_id IN 
      (
         '4411e09a-c8b7-4a2e-ae6e-5d028e303791'  -- DAL - Trinity Groves (Commerce)
      )
   THEN
      'Dallas'
   WHEN
      facility_id IN 
      (
         'ee395d7f-589b-4208-a745-90a8a0aa3bfd'  -- San Antonio - Lombrano
      )
   THEN
      'San Antonio'
   WHEN
      facility_id IN 
      (
         'ce1e2ca1-6944-48d4-a93c-67bf011560f6'  -- ATX - North Loop (Hwy 35)
      )
   THEN
      'Austin'
   WHEN
      facility_id IN 
      (
         '6f24178a-58f3-4d0a-a2c3-107ad22dcd4e',  -- ATL - Midtown (Forrest)
         '288c069f-0d07-4d56-8d10-dc8afab2cb94'  -- ATL - West End (Oak)
      )
   THEN
      'Georgia'
   WHEN
      facility_id IN 
      (
         '305dca7a-ffcf-4ecf-9363-4238af3cba6b',  -- PHL - University City (Fairmount)
         'efa863fe-4cf4-4552-b73d-253470878e8f'  -- PHL - North City Center (1308 W Girard)
      )
   THEN
      'Pennsylvania'
   WHEN
      facility_id IN 
      (
         'e92e292a-4ca8-4cbd-8b7d-82d275637c1f'  -- New Jersey - Belleville
      )
   THEN
      'New Jersey'
   WHEN
      facility_id IN 
      (
         'e380140f-dfa9-4b17-84aa-61af20324195'  -- PROV - Smith Hill (Bath)
      )
   THEN
      'Providence'
   WHEN
      facility_id IN 
      (
         '6a896a7e-ad07-4f06-87e9-1af5ea87d6cb',  -- NYC - Long Island City
         '65820060-3a5f-4561-9f7a-9a7e34c581c1',  -- NYC - Brooklyn (Utica)
         '9854b8e1-8c33-4ee0-b6b5-4ccbc2012a38',  -- NYC - Astoria (24-26 47th)
         '9f5e4667-6a63-4af6-a805-476c979884e2',  -- NY - The Bronx (Bullard Ave)
         '9209f242-c27f-4174-b941-73e4f7b00e56',  -- NA-US-NY-NYC-Broadway 
         '05db32ee-158c-4baa-97c6-402bba3fcad3',  -- NA-US-NY-NYC-Church
         '628e10de-20b0-400b-a006-8e023e4f5c4d',  -- NA-US-NY-Brooklyn-Commerce
         '7d62cf8a-dfc8-4988-a02e-0b27303b19e2'  -- NA-US-NY-NYC-Devoe
      )
   THEN
      'New York City'
   WHEN
      facility_id IN 
      (
         '8365d2e5-dd91-4a87-beaa-cb54c374acdc'  -- NA-US-NY-BUF-Military
      )
   THEN
      'Buffalo'
    WHEN
      facility_id IN 
      (
         '54d4375f-4c79-4444-ad83-f54362c08198'  -- NA-US-DC-5th
      )
   THEN
      'DC'
END
) AS city,
(
CASE
   WHEN facility_id = '2019e0f4-7fd8-47fd-a858-9e5c4b409a39' THEN 'NA-US-CO-DEN-Aurora'
   WHEN facility_id = 'ab647486-a823-456e-8058-053d052106de' THEN 'NA-US-CO-DEN-GRAPE'
   WHEN facility_id = '1e55ab2f-64a3-498c-a692-9614b79ab834' THEN 'NA-US-CO-DEN-Vallejo'
   
   WHEN facility_id = 'b0629713-f1bd-4d2b-a065-46ff82645987' THEN 'NA-US-CA(N)-SF-Charter Oak'
   WHEN facility_id = '7d248b5d-c113-4269-8f4a-68d155a48adf' THEN 'NA-US-CA(N)-SF-SoMa'
   WHEN facility_id = '983bc311-51e8-4e2e-ba27-6cc94bec511c' THEN 'NA-US-CA(N)-SF-Macarthur (Redwood City)'

   WHEN facility_id = '5d574e15-e1d4-42a3-837a-71eebe29728a' THEN 'NA-US-CA(N)-OAK-Adeline'
   WHEN facility_id = 'bc9dcc91-8fec-4a62-8438-024a4f65e8e8' THEN 'NA-US-CA(N)-OAK-Jingletown'

   WHEN facility_id = '905cbe43-327f-4d3e-a897-48ee81a64ade' THEN 'NA-US-CA(N)-SJ-Ruff (Kitchens)'

   WHEN facility_id = '557194c1-5796-469c-b3e0-d83736880340' THEN 'NA-US-CA(N)-SM-21st'

   WHEN facility_id = '9c7b97b6-878f-4007-9032-558c4c6157e9' THEN 'NA-US-CA(S)-COA-Lewis'

   WHEN facility_id = 'f0029277-0e7e-410a-ad4e-5247b892a280' THEN 'NA-US-CA(S)-LA-Hollywood'
   WHEN facility_id = '99573ac0-a3f2-4c70-946c-e9c06b425d5c' THEN 'NA-US-CA(S)-LA-Koreatown'
   WHEN facility_id = '11bcb365-1910-40e7-a25a-4d3dd15f6eb2' THEN 'NA-US-CA(S)-LAW-Hawthorne'
   WHEN facility_id = '5315a66f-8337-4c68-af42-86089a86f8ba' THEN 'NA-US-CA(S)-LA-LongBeach'
   WHEN facility_id = '55a786a5-1a9d-4611-95e2-81009b44ebdb' THEN 'NA-US-CA(S)-LA-Pico-SA'
   WHEN facility_id = '9732b09c-9d18-4f8e-a5d4-34173ddf1ea4' THEN 'NA-US-CA(S)-CC-Selmaraine'
   WHEN facility_id = '5efa5206-dc53-4ab4-9f49-e5a5a4dc1c34' THEN 'NA-US-CA(S)-LA-UniversityPark'

   WHEN facility_id = '4fff0653-ae90-4e98-a67d-f67001e33c65' THEN 'NA-US-CA(S)-SD-El Cajon'
   WHEN facility_id = 'c5c4ed02-1518-4970-a68f-432c75ecf6e2' THEN 'NA-US-CA(S)-SD-Downtown'

   WHEN facility_id = '2fe8235d-fe85-46e1-be54-023a65e34939' THEN 'NA-US-NV-LAS-North Strip'

   WHEN facility_id = '2d661ae4-d4bf-4d88-8af1-2df4f7d0fd75' THEN 'AZ - Tucson (E Delano ST)'

   WHEN facility_id = '937ddb7e-4c76-4cfd-bbb8-bd07b33ee7e9' THEN 'NA-US-AZ-PHX-Highland (Kitchens)'
   WHEN facility_id = 'a9eb0fc1-7699-43b4-bd4c-6bb5b8195a87' THEN 'NA-US-AZ-PHX-Tempe (Kitchens)'

   WHEN facility_id = '54341152-da81-4b58-8ba3-8604bc1ff2dc' THEN 'NA-US-IL-CHI-Avondale'
   WHEN facility_id = '0f9ac987-32ac-4b5a-91bf-78cbc7ea44bb' THEN 'NA-US-IL-CHI-Humboldt Park'
   WHEN facility_id = '75187d2a-01d8-467f-9993-90e200a3a586' THEN 'NA-US-IL-CHI-River West'
   WHEN facility_id = '6ab373fb-ee91-4dc6-b529-ef67f6326588' THEN 'NA-US-IL-CHI-Rockwell'
   WHEN facility_id = '94e1cb0b-facf-4f72-b66d-2bd97d98e58f' THEN 'NA-US-IL-CHI-South Loop'

   WHEN facility_id = '0a25ff7d-1697-4e0c-a394-71347b0cdcc3' THEN 'NA-US-MN-MPLS-Blaine'
   WHEN facility_id = '99174fc4-d9f4-424f-96d8-e499a85e85b6' THEN 'NA-US-MN-MPLS-Currie'

   WHEN facility_id = '4f7010ef-ba20-4a6a-927b-d1a37903ef5e' THEN 'NA-US-OH-Carnegie'

   WHEN facility_id = '6936d9f5-887a-4753-9c77-029c0164d956' THEN 'NA-US-OH-COL-ShortNorth (Kitchens)'
   
   WHEN facility_id = 'e1198539-7644-4c6d-9def-e64fa1cf4fb5' THEN 'NA-US-FL-MIA-Alton'
   WHEN facility_id = 'd5f67e55-66be-4742-81c1-4720bb6906d8' THEN 'ORL - West Downtown (Dollins)' -- pending SFDC Name

   WHEN facility_id = 'd1814be1-e541-484d-9fff-9a4d800bfa26' THEN 'NA-US-FL-ORL-Dollins'

   WHEN facility_id = 'ba4cd06d-e256-4562-8ffc-372c7a2ead8c' THEN 'NA-US-TX-HOU-Blodgett (Kitchen)'
   WHEN facility_id = '533a8386-c2a0-4e67-b6a3-ecea769ff764' THEN 'NA-US-TX-HOU-Fairdale (Kitchens)'

   WHEN facility_id = '68907756-0bb0-408f-a45c-3ab38083a57d' THEN 'NA-US-TX-FortWorth-CullenSt'
   WHEN facility_id = '2e30d25a-5c6b-46bf-b3f6-2030008dd5fd' THEN 'DFW - Towerwood (3230 Towerwood)' -- pending SFDC Name

   WHEN facility_id = '4411e09a-c8b7-4a2e-ae6e-5d028e303791' THEN 'NA-US-TX-DAL-Commerce'

   WHEN facility_id = 'ee395d7f-589b-4208-a745-90a8a0aa3bfd' THEN 'NA-US-TX-SA-Lombrano'

   WHEN facility_id = 'ce1e2ca1-6944-48d4-a93c-67bf011560f6' THEN 'NA-US-TX-AUS-North Loop'

   WHEN facility_id = '6f24178a-58f3-4d0a-a2c3-107ad22dcd4e' THEN 'NA-US-GA-ATL-Forrest (Kitchens)'
   WHEN facility_id = '288c069f-0d07-4d56-8d10-dc8afab2cb94' THEN 'NA-US-GA-ATL-Oak'

   WHEN facility_id = '305dca7a-ffcf-4ecf-9363-4238af3cba6b' THEN 'NA-US-PA-PHL-Fairmount'
   WHEN facility_id = 'efa863fe-4cf4-4552-b73d-253470878e8f' THEN 'NA-US-PA-PHL-Girard'

   WHEN facility_id = 'e92e292a-4ca8-4cbd-8b7d-82d275637c1f' THEN 'NA-US-NJ-Cortlandt'

   WHEN facility_id = 'e380140f-dfa9-4b17-84aa-61af20324195' THEN 'NA-US-RI-PROV-Smithhill'

   WHEN facility_id = '6a896a7e-ad07-4f06-87e9-1af5ea87d6cb' THEN 'NA-US-NY-LIC-Skillman'
   WHEN facility_id = '65820060-3a5f-4561-9f7a-9a7e34c581c1' THEN 'NA-US-NY-NYC-Utica'
   WHEN facility_id = '9854b8e1-8c33-4ee0-b6b5-4ccbc2012a38' THEN 'NA-US-IL-CHI-River West'
   WHEN facility_id = '9f5e4667-6a63-4af6-a805-476c979884e2' THEN 'NY - The Bronx (Bullard Ave)' -- pending SFDC Name
   WHEN facility_id = '9209f242-c27f-4174-b941-73e4f7b00e56' THEN 'NA-US-NY-NYC-Broadway'
   WHEN facility_id = '05db32ee-158c-4baa-97c6-402bba3fcad3' THEN 'NA-US-NY-NYC-Church'
   WHEN facility_id = '628e10de-20b0-400b-a006-8e023e4f5c4d' THEN 'NA-US-NY-Brooklyn-Commerce'
   WHEN facility_id = '7d62cf8a-dfc8-4988-a02e-0b27303b19e2' THEN 'NA-US-NY-NYC-Devoe'

   WHEN facility_id = '8365d2e5-dd91-4a87-beaa-cb54c374acdc' THEN 'NA-US-NY-BUF-Military'

   WHEN facility_id = '54d4375f-4c79-4444-ad83-f54362c08198' THEN 'NA-US-DC-5th'

END
) AS facility_sf_name,
facility_name,
organization_id,
co.order_id,
facility_id,
order_display_id,
ofo_slug, store_link_service_slug,
facility_country_code as country_code, 
is_cancelled,
oc_order_issue_count,
order_issue_details,
hoi_order_issue_details,
oc_order_issue_details,
order_issue_types,
hoi_order_issue_types,
oc_order_issue_types,

subtotal,
total,

commission, discount,	tip, tip_for_restaurant, tip_for_courier, delivery_fee, marketing_fee, service_fee

,ROW_NUMBER() OVER (PARTITION BY ofo_slug, external_order_id, order_items_quantity, subtotal ORDER BY co.order_id) as row_number

,cast(local_accepted_at_last_ts as TIME ) as accepted_local_time,
local_processed_at_ts,local_ordered_at_ts,local_accepted_at_ts, local_in_handoff_at_ts,
local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,local_pickup_picked_up_at_ts,
local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,fulfillment_state,fulfillment_mode,
pickup_state,scheduling_type,ofo_status,order_status
,cancellation_state,cancellation_reason as facility_cancellation_reason
,ofo_cancellation_reason,order_rating, order_review_full_text,location_state,

case when location_state='LOCATION_HANDED_OFF' then 1
  else 0
  end
as is_handed_off_num,

Case When in_handoff_at IS NOT NULL THEN 1
  else 0
  END 
as is_in_handoff_num,




CASE
        WHEN is_cancelled THEN 'cancel'
        WHEN order_issue_types IS NOT NULL THEN 'issue'
        ELSE 'success'
        END AS defect_type,

CASE
        WHEN is_cancelled AND cancellation_reason = 'UNAVAILABLE_ITEM' THEN 'item_unavailable'
        WHEN is_cancelled AND confirmation_state <> 'CONFIRMATION_CONFIRMED' THEN 'confirmation_failed'
        WHEN is_cancelled AND  location_state!='LOCATION_HANDED_OFF' THEN 'handoff_failed'
        WHEN is_cancelled AND time_to_last_in_handoff_millis > 1200000 THEN 'pick_over_20' --Pick time was over 20 minutes long
        WHEN is_cancelled THEN 'other_cancel'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ORDER%' THEN 'order_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%MISSING_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_QUANTITY%' THEN 'item_missing'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_SIZE%' THEN 'item_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%INCOMPLETE%' THEN 'order_incomplete'
        ELSE 'other_issue'
      END AS defect_reason,

COALESCE(IF(adjustment IS NOT NULL AND (is_cancelled OR order_issue_types IS NOT NULL), adjustment < 0), FALSE) AS is_refunds
,time_to_last_in_handoff_millis BETWEEN 1 AND 1200000 AS order_20picked --TRUE if picked in under 20 minutes

,order_source, hoi_only_order_type,order_items_quantity,scan_out.order_scanned_out,


printed_at_ts, in_handoff_at_ts, handed_off_at_ts, courier_checked_in_at_ts, courier_dropped_off_at_ts, courier_arrived_at_ts

,scanned_items

,customer_note

,concat('https://tools.cloudkitchens.com/order-inspector/order-details/',co.order_id,'/overview') as link


FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'USC'
   
left join scan_out
on co.order_id=scan_out.order_id






--aqui se filtra la fecha    
WHERE co.day_partition >= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '12' month, '%Y-%m-%d')

AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-15'
AND tenant_type = 'VIRTUAL'

--AND is_cancelled

AND facility_id IN ( -- 
-- COLORADO
    -- DENVER
           '2019e0f4-7fd8-47fd-a858-9e5c4b409a39',  -- DEN -  AURORA
           'ab647486-a823-456e-8058-053d052106de',   -- DEN - GRAPE
           '1e55ab2f-64a3-498c-a692-9614b79ab834',   -- DEN - VALLEJO

-- CALIFORNIA
    -- SAN FRANCISCO
           'b0629713-f1bd-4d2b-a065-46ff82645987',  -- SF - CHARTER OAK
           '7d248b5d-c113-4269-8f4a-68d155a48adf',  -- SF - SoMa
           '983bc311-51e8-4e2e-ba27-6cc94bec511c',  -- SF - Redwood
    -- OAKLAND
           '5d574e15-e1d4-42a3-837a-71eebe29728a',  -- OAK - Adeline
           'bc9dcc91-8fec-4a62-8438-024a4f65e8e8',  -- OAK - Jingletown (12th St)
   -- SAN JOSE
           '905cbe43-327f-4d3e-a897-48ee81a64ade',  -- SJO - Ruff (949 Ruff)
   -- SAN MATEO
           '557194c1-5796-469c-b3e0-d83736880340',  -- SF - San Mateo (21st Street)
   -- ANAHEIM
           '9c7b97b6-878f-4007-9032-558c4c6157e9',  -- LA - Anaheim (Lewis)
   -- LOS ANGELES
           'f0029277-0e7e-410a-ad4e-5247b892a280',  -- LA - Hollywood (Western)
           '99573ac0-a3f2-4c70-946c-e9c06b425d5c',  -- LA - Pico Union (1842 Washington)
           '11bcb365-1910-40e7-a25a-4d3dd15f6eb2',  -- LA - South Bay (Hawthorne)
           '5315a66f-8337-4c68-af42-86089a86f8ba',  -- LA - Long Beach (Daisy)
           '55a786a5-1a9d-4611-95e2-81009b44ebdb',  -- NA-US-CA(S)-LA-Pico-SA
           '9732b09c-9d18-4f8e-a5d4-34173ddf1ea4',  -- LA - Culver City (Selmaraine)
           '5efa5206-dc53-4ab4-9f49-e5a5a4dc1c34',  -- LA - University Park (W 38th)
    -- SAN DIEGO
           '4fff0653-ae90-4e98-a67d-f67001e33c65',  -- SD - College Area (El Cajon)
           'c5c4ed02-1518-4970-a68f-432c75ecf6e2',  -- SD - Downtown (Boston)
-- NEVADA
   -- LAS VEGAS
           '2fe8235d-fe85-46e1-be54-023a65e34939',  -- LAS - North Strip (St Louis)
-- ARIZONA
   -- TUCSON
           '2d661ae4-d4bf-4d88-8af1-2df4f7d0fd75',  -- AZ - Tucson (E Delano ST)
   -- PHOENIX
           '937ddb7e-4c76-4cfd-bbb8-bd07b33ee7e9',  -- PHX - Midtown (W Highland)
           'a9eb0fc1-7699-43b4-bd4c-6bb5b8195a87',  -- PHX - Tempe (E 5th)
-- ILLINOIS
   -- TUCSON
           '54341152-da81-4b58-8ba3-8604bc1ff2dc',  -- CHI - Avondale (Spaulding)
           '0f9ac987-32ac-4b5a-91bf-78cbc7ea44bb',  -- CHI - Humboldt Park (Grand)
           '75187d2a-01d8-467f-9993-90e200a3a586',  -- CHI - River West (Superior)
           '6ab373fb-ee91-4dc6-b529-ef67f6326588',  -- CHI - North Center (Rockwell)
           '94e1cb0b-facf-4f72-b66d-2bd97d98e58f',  -- CHI - South Loop (Wabash)
-- MINNESOTA
   -- MINNEAPOLIS
           '0a25ff7d-1697-4e0c-a394-71347b0cdcc3',  -- Minneapolis - Blaine
           '99174fc4-d9f4-424f-96d8-e499a85e85b6',  -- Minneapolis - Currie Avenue
-- OHIO
   -- CLEVELAND
           '4f7010ef-ba20-4a6a-927b-d1a37903ef5e',  -- OH - Cleveland - Central (Carnegie Ave)
   -- COLUMBUS
           '6936d9f5-887a-4753-9c77-029c0164d956',  -- COL - Short North (Essex)
-- FLORIDA
   -- MIAMI
           'e1198539-7644-4c6d-9def-e64fa1cf4fb5',  -- MIA - South Beach (Alton)
           'd5f67e55-66be-4742-81c1-4720bb6906d8',  -- MIA - Wynwood (Miami Ct)
   -- ORLANDO
           'd1814be1-e541-484d-9fff-9a4d800bfa26',  -- ORL - West Downtown (Dollins)
-- TEXAS
   -- HOUSTON
           'ba4cd06d-e256-4562-8ffc-372c7a2ead8c',  -- HOU - Midtown (Blodgett)
           '533a8386-c2a0-4e67-b6a3-ecea769ff764',  -- HOU - Galleria (Fairdale)
   -- FORTWORTH
           '68907756-0bb0-408f-a45c-3ab38083a57d',  -- DFW - Cullen (3004 Cullen)
           '2e30d25a-5c6b-46bf-b3f6-2030008dd5fd',  -- DFW - Towerwood (3230 Towerwood)
   -- DALLAS
           '4411e09a-c8b7-4a2e-ae6e-5d028e303791',  -- DAL - Trinity Groves (Commerce)
   -- SAN ANTONIO
           'ee395d7f-589b-4208-a745-90a8a0aa3bfd',  -- San Antonio - Lombrano
   -- AUSTIN
           'ce1e2ca1-6944-48d4-a93c-67bf011560f6',  -- ATX - North Loop (Hwy 35)
-- ATLANTA
   -- GEORGIA
           '6f24178a-58f3-4d0a-a2c3-107ad22dcd4e',  -- ATL - Midtown (Forrest)
           '288c069f-0d07-4d56-8d10-dc8afab2cb94',  -- ATL - West End (Oak)
-- PHILADELPHIA
   -- PENNSYLVANIA
           '305dca7a-ffcf-4ecf-9363-4238af3cba6b',  -- PHL - University City (Fairmount)
           'efa863fe-4cf4-4552-b73d-253470878e8f',  -- PHL - North City Center (1308 W Girard)
-- NEW JERSEY
   -- NEW JERSEY
           'e92e292a-4ca8-4cbd-8b7d-82d275637c1f',  -- New Jersey - Belleville
-- RHODE ISLAND
   -- PROVIDENCE
           'e380140f-dfa9-4b17-84aa-61af20324195',  -- PROV - Smith Hill (Bath)
-- NEW YORK
    -- NEW YORK CITY
           '6a896a7e-ad07-4f06-87e9-1af5ea87d6cb',  -- NYC - Long Island City
           '65820060-3a5f-4561-9f7a-9a7e34c581c1',  -- NYC - Brooklyn (Utica)
           '9854b8e1-8c33-4ee0-b6b5-4ccbc2012a38',  -- NYC - Astoria (24-26 47th)
           '9f5e4667-6a63-4af6-a805-476c979884e2',  -- NY - The Bronx (Bullard Ave)
           '9209f242-c27f-4174-b941-73e4f7b00e56',  -- NA-US-NY-NYC-Broadway
           '05db32ee-158c-4baa-97c6-402bba3fcad3',  -- NA-US-NY-NYC-Church
           '628e10de-20b0-400b-a006-8e023e4f5c4d',  -- NA-US-NY-Brooklyn-Commerce
           '7d62cf8a-dfc8-4988-a02e-0b27303b19e2',  -- NA-US-NY-NYC-Devoe
    -- BUFFALO
           '8365d2e5-dd91-4a87-beaa-cb54c374acdc',  -- NA-US-NY-BUF-Military
-- WASHINGTON DC
    -- WASHINGTON DC
           '54d4375f-4c79-4444-ad83-f54362c08198'  -- NA-US-DC-5th
                
                )


)


Select created_month,country, ordenes.organization_id, ordenes.facility_id, ordenes.facility_sf_name

,CASE 
when org.deleted_at is not null then True

ELSE  False END

as org_deleted_skusku





, count (DISTINCT (order_id)) as total_orders
, count(DISTINCT CASE WHEN is_cancelled THEN order_id END) as cancelled_orders
,count (DISTINCT CASE WHEN defect_type = 'issue' THEN order_id END) as issue_orders
,count (DISTINCT CASE WHEN defect_type != 'success' THEN order_id END) as ODR_orders
,count(DISTINCT  CASE WHEN order_scanned_out THEN order_id END) as scanned_orders
,sum(subtotal) as total_gmv_local
,SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS cancelled_gmv_local
,count (DISTINCT ofo_slug) as OFOs
,count(DISTINCT city) as cities
, count(DISTINCT facility_sf_name) as facilities
, AVG (order_items_quantity) as Avg_Items_per_Order
, AVG(order_rating) as avg_rating
, AVG(in_handoff_at_ts-printed_at_ts) as avg_picking_time
,  avg(date_diff('minute',printed_at_ts,in_handoff_at_ts) )as avg_picking_time_minutes 
from ordenes

left join hudi_ingest.processed.store_config_organization_view as org

on ordenes.organization_id=org.organization_id

WHERE  row_number=1



group by 1,2,3,4,5,6

"""
print("Querying data...")
opp_data = ds_trino.fetch_data(conn=conn, sql_str= opty_query, use_cache=False)
print("Data pulled!")
print(len(opp_data))

Querying data...
Data pulled!
6844


In [13]:
opp_data.head()

,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes
0,2021-12-01,United States,8a0b24e8-605f-4071-a2e3-42b6f23a8190,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,132,33,3,36,0,4020.52,926.14,2,1,1,2.075758,4.833333,0 00:08:40.058,8.162791
1,2022-11-01,United States,5e2e3c62-54dc-4fbd-9af2-a423eefa56cb,bc9dcc91-8fec-4a62-8438-024a4f65e8e8,NA-US-CA(N)-OAK-Jingletown,False,24,4,1,5,0,588.25,130.00,2,1,1,7.541667,NaN,None,NaN
2,2022-08-01,United States,de4e398c-8436-4a1f-bfb0-c913de23e98f,f0029277-0e7e-410a-ad4e-5247b892a280,NA-US-CA(S)-LA-Hollywood,False,15,1,0,1,14,696.00,66.00,1,1,1,1.733333,NaN,-0 00:01:29.714,-1.928571
3,2022-08-01,United States,34d772ee-98fb-442b-ac0a-2258c66be88e,ee395d7f-589b-4208-a745-90a8a0aa3bfd,NA-US-TX-SA-Lombrano,False,7,1,0,1,0,112.00,24.00,3,1,1,2.000000,NaN,0 00:02:09.429,1.571429
4,2022-10-01,United States,279612da-8612-4951-98da-63477061dc36,557194c1-5796-469c-b3e0-d83736880340,NA-US-CA(N)-SM-21st,False,22,0,0,0,0,910.00,0.00,2,1,1,2.772727,5.000000,0 00:02:15.500,1.750000


In [15]:
# Inspect missing data
print("missing data as % of all data")
opp_data.isna().sum() / len(gmv_data)

missing data as % of all data


created_month               0.000000
country                     0.000000
organization_id             0.000000
facility_id                 0.000000
facility_sf_name            0.000000
org_deleted_skusku          0.000000
total_orders                0.000000
cancelled_orders            0.000000
issue_orders                0.000000
ODR_orders                  0.000000
scanned_orders              0.000000
total_gmv_local             0.001029
cancelled_gmv_local         0.000000
OFOs                        0.000000
cities                      0.000000
facilities                  0.000000
Avg_Items_per_Order         0.000000
avg_rating                  0.647870
avg_picking_time            0.412019
avg_picking_time_minutes    0.412019
dtype: float64

In [21]:
# Inspect a single customer
opp_data[opp_data["organization_id"] == "8a0b24e8-605f-4071-a2e3-42b6f23a8190"].facility_sf_name.unique()

array(['NA-US-CA(S)-LA-Koreatown', 'NA-US-OH-COL-ShortNorth (Kitchens)',
       'NA-US-TX-HOU-Blodgett (Kitchen)', 'NA-US-CA(N)-OAK-Jingletown',
       'NA-US-FL-ORL-Dollins'], dtype=object)